##Reduce / transform Census Tract data
[Data](https://docs.google.com/file/d/0B1aa6nX82m2WY2RBNERsd0VfN1k/edit)

In [27]:
import pandas as pd

In [28]:
path_base = ("/Users/brian/Google Drive/")
path_specfic = "SPHIP/2009-2013 ACS Demographic Data/Census Tract Data Files and Documentation/"

In [29]:
file_name = 'ACS_13_5YR_DP02.txt'
with open(path_base+path_specfic+file_name) as f:
    text_description = f.read()
print(text_description[:1000])

DP02
SELECTED SOCIAL CHARACTERISTICS IN THE UNITED STATES

Although the American Community Survey (ACS) produces population, demographic and housing unit estimates, it is the Census Bureau's Population Estimates Program that produces and disseminates the official estimates of the population for the nation, states, counties, cities and towns and estimates of housing units for states and counties.


Supporting documentation on code lists, subject definitions, data accuracy, and statistical testing can be found on the American Community Survey website in the Data and Documentation section.

Sample size and data quality measures (including coverage rates, allocation rates, and response rates) can be found on the American Community Survey website in the Methodology section.


Source:  U.S. Census Bureau, 2009-2013 5-Year American Community Survey


Explanation of Symbols:An '**' entry in the margin of error column indicates that either no sample observations or too few sample observations w

In [30]:
# Load meta data
file_name = "ACS_13_5YR_DP02_metadata.csv"
path_name = (path_base+path_specfic+file_name)
df_meta = pd.read_csv(path_name)
df_meta.head()

,GEO.id,Id
0,GEO.id2,Id2
1,GEO.display-label,Geography
2,HC01_VC03,Estimate; HOUSEHOLDS BY TYPE - Total households
3,HC02_VC03,Margin of Error; HOUSEHOLDS BY TYPE - Total ho...
4,HC03_VC03,Percent; HOUSEHOLDS BY TYPE - Total households


In [35]:
# Load data
file_name = "ACS_13_5YR_DP02_with_ann.csv"
path_name = (path_base+path_specfic+file_name)
df_data_with_ann = pd.read_csv(path_name)
df_data_with_ann.head(n=2)

,GEO.id,GEO.id2,GEO.display-label,HC01_VC03,HC02_VC03,HC03_VC03,HC04_VC03,HC01_VC04,HC02_VC04,HC03_VC04,...,HC03_VC216,HC04_VC216,HC01_VC217,HC02_VC217,HC03_VC217,HC04_VC217,HC01_VC218,HC02_VC218,HC03_VC218,HC04_VC218
0,Id,Id2,Geography,Estimate; HOUSEHOLDS BY TYPE - Total households,Margin of Error; HOUSEHOLDS BY TYPE - Total ho...,Percent; HOUSEHOLDS BY TYPE - Total households,Percent Margin of Error; HOUSEHOLDS BY TYPE - ...,Estimate; HOUSEHOLDS BY TYPE - Total household...,Margin of Error; HOUSEHOLDS BY TYPE - Total ho...,Percent; HOUSEHOLDS BY TYPE - Total households...,...,Percent; COMPUTERS AND INTERNET USE - Total Ho...,Percent Margin of Error; COMPUTERS AND INTERNE...,Estimate; COMPUTERS AND INTERNET USE - Total H...,Margin of Error; COMPUTERS AND INTERNET USE - ...,Percent; COMPUTERS AND INTERNET USE - Total Ho...,Percent Margin of Error; COMPUTERS AND INTERNE...,Estimate; COMPUTERS AND INTERNET USE - Total H...,Margin of Error; COMPUTERS AND INTERNET USE - ...,Percent; COMPUTERS AND INTERNET USE - Total Ho...,Percent Margin of Error; COMPUTERS AND INTERNE...
1,1400000US06075010100,06075010100,"Census Tract 101, San Francisco County, Califo...",2177,126,2177,(X),603,137,27.7,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [36]:
# 1st row is annotations / meta data
# annotations = df_data_with_ann[0,:]

Ideas and work is tracked on [Hackpad](https://datakindsfbayarea.hackpad.com/SF-Health-Improvement-Partnership-SFHIP-IdGfO4Yn60V)

In [ ]:
# Filter / reduce data

In [ ]:
# Define these groups:
ACS_13_5YR_DP02_metadata
Total households
Families with children 
Under 12
12 to 18
19 to 64
65 and over
Single Parents
Under 12
12 to 18
19 to 64
65 and over
Average family size
Estimate; HOUSEHOLDS BY TYPE - Average family size
Educational Attainment
No Educational Attainment
High School
Collect
Master and Beyond
Disability
Percent; DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION - 18 to 64 years
Percent; PLACE OF BIRTH - Total population - Native